In [1]:
from EMDA import EMDA, __version__
print("EMDA version is:", __version__)


from numpy import average

import matplotlib.pyplot as plt

plt.style.use('ggplot')

EMDA version is: 1.0.0a3


# Load parameters and trajectory <a class="anchor" id="load"></a>

In [2]:
parameters = 'parameters.prmtop' # AMBER parameters file
trajectory = 'trajectory.nc'     # AMBER NetCDF coordinates (10 frames)

In [3]:
emda = EMDA(parameters=parameters, trajectory=trajectory)

Trajectory has been loaded!


In [4]:
emda.load_trajectory(trajectory)
emda.load_variant(parameters, trajectory)
#emda.load_trajectory(trajectory, variant_name='V1')
emda.load_trajectory(trajectory)

emda.universe

A new replica has been loaded to variant V1!
V2 variant has been loaded!
A new replica has been loaded to variant V2!


{'V1': {'R1': <Universe with 79250 atoms>, 'R2': <Universe with 79250 atoms>},
 'V2': {'R1': <Universe with 79250 atoms>, 'R2': <Universe with 79250 atoms>}}

# Select the atoms to be used for the analysis <a class="anchor" id="select"></a>

In [5]:
#emda.select('C10', 'C10', sel_type='at_name',)
#emda.select('C11', 'C11', sel_type='at_name',)
emda.select('C12', 'C12', sel_type='at_name',)
#emda.select('C13', 'C13', sel_type='at_name')
#emda.select('C14', 'C14', sel_type='at_name',)
emda.select('H12', ['H12A', 'H12B'], sel_type='at_name')
emda.select('cof', 10597, sel_type='at_num')

emda.select('COO', [10599, 10600, 10601], sel_type='at_num')

#subs = emda.universe.select_atoms('resid 666')

print(emda.selections)

{'C12': 'name C12', 'H12': 'name H12A or name H12B', 'cof': 'bynum 10597', 'COO': 'bynum 10599 or bynum 10600 or bynum 10601'}


# Analysis of the trajectory <a class="anchor" id="trajanalysis"></a>

In [6]:
?emda.add_protein_contacts

Signature:
emda.add_protein_contacts(
    name,
    sel_env=3,
    include_WAT=False,
    measure_distances=False,
)
Docstring:
DESCRIPTION:
    This function takes a adds the measure of the contacts of all residues in a protein

INPUT:
    - Name of the measurement
    - sel_env      -> radius (in ang) around each residue
    - 
    - interactions -> type of interactions to be considered (all, polar, nonpolar, donorHbond, none). Custom
                        interactions can be also analysed by passing a list of residues names


OUTPUT:
    - List of dictionaries containing the name and number of all interacting residues
File:      ~/miniconda3/envs/emda-dev/lib/python3.12/site-packages/EMDA/adders.py
Type:      method

In [7]:
emda.add_distance('dist_H12', 'cof', 'H12')
#emda.add_angle('angle_C11C12C13', 'C11', 'C12', 'C13')
#emda.add_dihedral('dihe_C10C11C13C14', 'C10', 'C11', 'C13', 'C14', domain=180)
#emda.add_contacts('contacts_COO', 'COO', sel_env=5, interactions='all', include_WAT=True, measure_distances=True)
emda.add_protein_contacts('contacts_prot', sel_env=3, measure_distances=True)
#emda.add_pKa('pka', excluded_ions=['Na+'], keep_pdb=True, keep_pka=True)
#emda.add_RMSD('RMSD_subs', subs)

## Run the measurements

In [8]:
print(emda.measures)

{'dist_H12': Measure dataclass with:
	Name:   dist_H12
	Type:   distance
	Sel:    ['cof', 'H12']
	Status: 
		V1, R1: Not calculated
		V1, R2: Not calculated
		V2, R1: Not calculated
		V2, R2: Not calculated
, 'contacts_prot': Measure dataclass with:
	Name:   contacts_prot
	Type:   protein_contacts
	Sel:    ['protein', 3]
	Status: 
		V1, R1: Not calculated
		V1, R2: Not calculated
		V2, R1: Not calculated
		V2, R2: Not calculated
}


In [9]:
emda.run(sleep_time=0)

Variants:   0%|          | 0/2 [00:00<?, ?var/s]

Starting variant V1 
Starting replica R1 (1 of 2)


Measuring variant V1, replica R1:   0%|          | 0/10 [00:00<?, ?Frame/s]

Starting replica R2 (2 of 2)


Measuring variant V1, replica R2:   0%|          | 0/10 [00:00<?, ?Frame/s]

Starting variant V2 
Starting replica R1 (1 of 2)


Measuring variant V2, replica R1:   0%|          | 0/10 [00:00<?, ?Frame/s]

Starting replica R2 (2 of 2)


Measuring variant V2, replica R2:   0%|          | 0/10 [00:00<?, ?Frame/s]

emda.measures['dist_H12'].result

## Analyse distances, angles and dihedrals

emda.analyse_value('dist_H12_bool', 'dist_H12', 3)

emda.analyses['dist_H12_bool'].result

print('angle_C11C12C13 average is:', average(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 min is:', min(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 max is:', max(emda.measures['angle_C11C12C13'].result))

emda.analyse_value('angle_C11C12C13_bool', 'angle_C11C12C13', 110, 5, mode='tol')

emda.analyses

emda.analyse_NACs('dist_angle_NACs', ['dist_H12_bool', 'angle_C11C12C13_bool'], inverse=False)


print('dist_H12_bool:\t\t\t', emda.analyses['dist_H12_bool'].result)
print('angle_C11C12C13_bool:\t\t', emda.analyses['angle_C11C12C13_bool'].result)
print('dist_angle_NACs:\t\t', emda.analyses['dist_angle_NACs'].result)

## Analyse contacts

emda.analyse_contacts_frequency('contacts_COO_freq', 'contacts_COO')

emda.analyses['contacts_COO_freq'].result['V1']['R2']

In [10]:
emda.analyse_contacts_frequency('contacts_prot_freq', 'contacts_prot', percentage=True, normalise_to_most_frequent=True)
#emda.analyse_contacts_amount('contacts_prot_amount', 'contacts_prot')

emda.analyses['contacts_prot_freq'].result['V1']['R1']

{'MET1': {'GLY2': 100.0,
  'PRO86': 30.0,
  'ALA54': 100.0,
  'GLU55': 100.0,
  'VAL53': 100.0,
  'ASP56': 100.0},
 'GLY2': {'PRO86': 90.0, 'VAL53': 100.0, 'ARG3': 100.0, 'TYR4': 70.0},
 'ARG3': {'ASP52': 100.0,
  'TYR4': 100.0,
  'PRO86': 90.0,
  'HID51': 100.0,
  'GLY85': 10.0,
  'ASP50': 100.0,
  'VAL53': 50.0},
 'TYR4': {'GLN84': 100.0,
  'LEU60': 80.0,
  'LEU57': 60.0,
  'GLY2': 70.0,
  'GLY85': 80.0,
  'HID51': 100.0,
  'PRO86': 50.0,
  'LEU26': 10.0,
  'VAL83': 100.0,
  'ASP50': 100.0,
  'VAL53': 100.0,
  'ARG5': 100.0},
 'ARG5': {'GLN84': 100.0,
  'PHE49': 90.0,
  'ILE6': 100.0,
  'VAL83': 100.0,
  'ASP50': 100.0,
  'GLU48': 10.0},
 'ILE6': {'LEU35': 100.0,
  'ARG7': 100.0,
  'ILE81': 90.0,
  'LEU26': 20.0,
  'PHE49': 100.0,
  'THR82': 100.0,
  'LEU37': 70.0,
  'LEU39': 50.0,
  'VAL8': 90.0,
  'VAL83': 100.0,
  'ASP50': 50.0,
  'LEU24': 100.0,
  'GLU48': 100.0},
 'ARG7': {'ILE81': 80.0,
  'THR82': 100.0,
  'GLU46': 100.0,
  'GLU47': 100.0,
  'VAL8': 100.0,
  'GLU48': 100.0,
  '

In [11]:
emda.analyses['contacts_prot_freq'].options

{'mode': 'protein_contacts', 'percentage': True}

In [12]:
emda.analyses['contacts_prot_freq'].result['V1']['R1']

{'MET1': {'GLY2': 100.0,
  'PRO86': 30.0,
  'ALA54': 100.0,
  'GLU55': 100.0,
  'VAL53': 100.0,
  'ASP56': 100.0},
 'GLY2': {'PRO86': 90.0, 'VAL53': 100.0, 'ARG3': 100.0, 'TYR4': 70.0},
 'ARG3': {'ASP52': 100.0,
  'TYR4': 100.0,
  'PRO86': 90.0,
  'HID51': 100.0,
  'GLY85': 10.0,
  'ASP50': 100.0,
  'VAL53': 50.0},
 'TYR4': {'GLN84': 100.0,
  'LEU60': 80.0,
  'LEU57': 60.0,
  'GLY2': 70.0,
  'GLY85': 80.0,
  'HID51': 100.0,
  'PRO86': 50.0,
  'LEU26': 10.0,
  'VAL83': 100.0,
  'ASP50': 100.0,
  'VAL53': 100.0,
  'ARG5': 100.0},
 'ARG5': {'GLN84': 100.0,
  'PHE49': 90.0,
  'ILE6': 100.0,
  'VAL83': 100.0,
  'ASP50': 100.0,
  'GLU48': 10.0},
 'ILE6': {'LEU35': 100.0,
  'ARG7': 100.0,
  'ILE81': 90.0,
  'LEU26': 20.0,
  'PHE49': 100.0,
  'THR82': 100.0,
  'LEU37': 70.0,
  'LEU39': 50.0,
  'VAL8': 90.0,
  'VAL83': 100.0,
  'ASP50': 50.0,
  'LEU24': 100.0,
  'GLU48': 100.0},
 'ARG7': {'ILE81': 80.0,
  'THR82': 100.0,
  'GLU46': 100.0,
  'GLU47': 100.0,
  'VAL8': 100.0,
  'GLU48': 100.0,
  '

In [13]:
print(emda.analyses['contacts_prot_freq'].result)
print(emda.analyses['contacts_prot_amount'].result)

{'V1': {'R1': {'MET1': {'ASP56': 10, 'ALA54': 10, 'VAL53': 10, 'PRO86': 3, 'GLY2': 10, 'GLU55': 10}, 'GLY2': {'TYR4': 7, 'VAL53': 10, 'ARG3': 10, 'PRO86': 9}, 'ARG3': {'TYR4': 10, 'HID51': 10, 'VAL53': 5, 'PRO86': 9, 'GLY85': 1, 'ASP50': 10, 'ASP52': 10}, 'TYR4': {'ARG5': 10, 'LEU57': 6, 'HID51': 10, 'VAL53': 10, 'GLN84': 10, 'LEU26': 1, 'LEU60': 8, 'PRO86': 5, 'GLY2': 7, 'VAL83': 10, 'GLY85': 8, 'ASP50': 10}, 'ARG5': {'GLN84': 10, 'PHE49': 9, 'VAL83': 10, 'ILE6': 10, 'ASP50': 10, 'GLU48': 1}, 'ILE6': {'THR82': 10, 'LEU35': 10, 'ILE81': 9, 'ARG7': 10, 'PHE49': 10, 'LEU26': 2, 'VAL8': 9, 'LEU39': 5, 'LEU37': 7, 'VAL83': 10, 'GLU48': 10, 'ASP50': 5, 'LEU24': 10}, 'ARG7': {'THR82': 10, 'ILE81': 8, 'GLU47': 10, 'VAL8': 10, 'GLU46': 10, 'ARG80': 1, 'GLU48': 10}, 'VAL8': {'THR10': 4, 'ILE81': 10, 'GLU46': 10, 'GLU47': 10, 'PRO41': 8, 'LEU39': 10, 'ALA9': 10, 'TRP76': 9, 'ILE6': 9, 'ARG80': 10, 'LEU24': 3}, 'ALA9': {'GLU45': 2, 'GLU46': 10, 'ASP79': 8, 'TRP76': 6, 'THR10': 10, 'ARG80': 10}, '

KeyError: 'contacts_prot_amount'

In [ ]:
from EMDA import ext_plot_contacts_frequencies_differences

In [ ]:
# Will output the bar plot if the two input lists are different
ext_plot_contacts_frequencies_differences(emda.analyses['contacts_prot_freq'].result, emda.analyses['contacts_prot_freq'].result)